In [1]:
%matplotlib inline
import random
import torch

In [2]:
def synthetic_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(X,w)+b
    y+=torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

In [3]:
true_w = torch.Tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([0.3149, 0.6291]) 
label: tensor([2.6751])


### 生成数据集
这里使用迭代器的方式来实现对数据集的构成，这样可以实现batch_size的数据集的获取

In [4]:
def data_iter(batch_size,features,labels):
    num_examples = len(features)
    indices = list(range(num_examples))

    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        batch_indices = torch.tensor(
            indices[i:min(i+batch_size,num_examples)]
        )
        yield features[batch_indices],labels[batch_indices]

In [5]:
batch_size = 10

for X,y in data_iter(batch_size,features,labels):
    print(X,'\n',y)
    break

tensor([[ 0.7817,  0.0851],
        [ 0.9564,  1.8865],
        [-1.1413, -0.0398],
        [-0.5371, -1.9656],
        [-0.6010, -0.7085],
        [-0.2328, -0.0915],
        [-0.8251, -0.2667],
        [ 1.7524,  0.9099],
        [-1.1060,  0.3712],
        [ 0.3577, -0.1501]]) 
 tensor([[ 5.4578],
        [-0.3003],
        [ 2.0510],
        [ 9.8192],
        [ 5.4087],
        [ 4.0467],
        [ 3.4615],
        [ 4.6039],
        [ 0.7394],
        [ 5.4329]])


In [7]:
# 参数的初始化
w = torch.normal(0,0.1,size=(2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [8]:
def linreg(X,w,b):
    return torch.matmul(X,w) + b

In [9]:
def squared_loss(y_hat,y):
    # 将真实值y的形状转换为和预测值y_hat的形状相同
    return (y_hat - y.reshape(y_hat.shape))**2/2

In [10]:
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad/batch_size
            param.grad.zero_()

In [11]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b),y)
        l.sum().backward()
        sgd([w,b],lr,batch_size)

    with torch.no_grad():
        train_l = loss(net(features,w,b),labels)
        print(f'epoch {epoch+1},loss {float(train_l.mean()):f}')

epoch 1,loss0.036918
epoch 2,loss0.000126
epoch 3,loss0.000047


In [12]:
print(f'w error:{true_w - w.reshape(true_w.shape)}')
print(f'b error:{true_b - b}')

w error:tensor([ 3.8981e-05, -3.4118e-04], grad_fn=<SubBackward0>)
b error:tensor([0.0001], grad_fn=<RsubBackward1>)
